# Image Segmentation with Breast Cancer Dataset using U-Net

In [2]:
import os 
import random
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
#import albumentations as A


## I used Kaggle Dataset and working in Kaggle 

In [ ]:
os.getcwd()

os.chdir("/kaggle/input/augmentedclear")
os.listdir()



## İmport Numpy files


In [ ]:
data = np.load("augmentedData_V2.npy")
label = np.load("augmentedLabel_V2.npy")

In [ ]:
print(label.shape)

### I have filtered images based on their tags. I used those with a ratio of white area on the label to the whole label greater than 0.005.

In [ ]:
# # n = 10
# # veri = data[n,:,:,0]
# # etiket = label[n,:,:,0]
# # plt.imshow(veri,cmap="Greys")
# # plt.figure()
# # plt.imshow(etiket,cmap="Greys")

# say = 0
# for i in range(label.shape[0]):
#     if np.sum(label[i,:,:,0])/(512*512)>0.005:
#         #print(np.sum(label[i,:,:,0])/(512*512))
#         say += 1
#         #print(i)
#    #plt.imshow(veri,cmap="Greys")
# print(say)

### adding filtered images to new numpy arrays

In [ ]:
# filteredLabel = np.zeros((1238,512,512,2),dtype="uint8")#(172,512,512,2)
# filteredData = np.zeros((1238,512,512,1),dtype="uint16")#(172,512,512,1)
# j = 0
# for i in range(label.shape[0]):
#     #print(np.sum(lbl[:,:,0]))
#     if (np.sum(label[i,:,:,0])/(512*512))>0.005:
#         filteredLabel[j,:,:,:] = label[i,:,:,:]
#         filteredData[j,:,:,:] = data[i,:,:,:]
#         j += 1

        

# Libraries

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt
from keras.models import model_from_json
import keras.models as models
from keras.metrics import MeanIoU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras import Input
from keras.layers import concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras import backend as K


## Unet architecture

In [ ]:
def get_unet_512(input_shape=(512, 512, 1),
                 num_classes=2):
    inputs = Input(shape=input_shape)
    # 512

    down0a = Conv2D(64, (3, 3), padding='same')(inputs)
    down0a = BatchNormalization()(down0a)
    down0a = Activation('relu')(down0a)
    down0a = Conv2D(64, (3, 3), padding='same')(down0a)
    down0a = BatchNormalization()(down0a)
    down0a = Activation('relu')(down0a)
    down0a_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0a)
    # 256

    down0 = Conv2D(128, (3, 3), padding='same')(down0a_pool)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0 = Conv2D(128, (3, 3), padding='same')(down0)
    down0 = BatchNormalization()(down0)
    down0 = Activation('relu')(down0)
    down0_pool = MaxPooling2D((2, 2), strides=(2, 2))(down0)
    # 128

    down1 = Conv2D(256, (3, 3), padding='same')(down0_pool)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1 = Conv2D(256, (3, 3), padding='same')(down1)
    down1 = BatchNormalization()(down1)
    down1 = Activation('relu')(down1)
    down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
    # 64

    down2 = Conv2D(512, (3, 3), padding='same')(down1_pool)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2 = Conv2D(512, (3, 3), padding='same')(down2)
    down2 = BatchNormalization()(down2)
    down2 = Activation('relu')(down2)
    down2_pool = MaxPooling2D((2, 2), strides=(2, 2))(down2)
    # 32

    center = Conv2D(1024, (3, 3), padding='same')(down2_pool)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    # center

    up4 = UpSampling2D((2, 2))(center)
    up4 = concatenate([down2, up4], axis=3)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    up4 = Conv2D(512, (3, 3), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Activation('relu')(up4)
    # 16

    up3 = UpSampling2D((2, 2))(up4)
    up3 = concatenate([down1, up3], axis=3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    up3 = Conv2D(256, (3, 3), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Activation('relu')(up3)
    # 32

    up2 = UpSampling2D((2, 2))(up3)
    up2 = concatenate([down0, up2], axis=3)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    up2 = Conv2D(128, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    # 64

    up1 = UpSampling2D((2, 2))(up2)
    up1 = concatenate([down0a, up1], axis=3)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    up1 = Conv2D(64, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    # 128

    # classify
    classify = Conv2D(num_classes, (1, 1), activation='sigmoid')(up1)

    model = Model(inputs=inputs, outputs=classify)

    return model

In [ ]:
os.chdir("/kaggle/working")
for i in os.listdir():
    os.remove(i)

In [ ]:
unet = get_unet_512()
os.chdir("/kaggle/working")

model_json = unet.to_json()
with open('unet.json', 'w') as json_file:
    json_file.write(model_json)


## I used early stop to avoid to overfitting.

In [ ]:
#os.mkdir("checkpoints")
os.chdir("/kaggle/working")
#checkpoint = ModelCheckpoint(fpath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, period=1)

fpath='unet_{epoch:02d}_{val_loss:.2f}.hdf5'
my_callbacks = [
    #patience ile kaç kere değişimin olmayacağını ayarlıyoruz (modelin sabrını ayarlıyoruz), 
    #monitor ise modeli ne üzerinden izleyeceğimiz. genelde val_loss alınır. acc veya val_acc alınmaz yanlış olur.
    EarlyStopping(patience=3,monitor="loss",),
    ModelCheckpoint(filepath=fpath,monitor="val_loss",verbose=1, save_best_only=False, save_weights_only=True, period=1),]


## Dice coef for loss function


In [ ]:
def dice(y_pred, y_true):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_loss(y_true, y_pred):
    return 1-dice(y_true, y_pred)

In [ ]:
optimizer = RMSprop(lr=3e-6)
unet.compile(loss=dice_loss,optimizer=optimizer, metrics=MeanIoU(num_classes=2))

In [ ]:
dataV1 = data/2**16


In [ ]:
labelV1 = label.astype(np.float32)

In [ ]:
x_train, x_test, x_val = dataV1[:1000],dataV1[1000:1100],dataV1[1100:]
y_train, y_test, y_val = labelV1[:1000],labelV1[1000:1100],labelV1[1100:]


In [ ]:
print(x_train.dtype)
print(y_val.shape)

In [ ]:
# os.chdir("/kaggle/working")
# for i in os.listdir():
#     print(i)
    
# json_file = open("unet.json", 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# unet = model_from_json(loaded_model_json)
# #unet.summary()
# unet.load_weights("unet_25_0.08.hdf5")


## Training

In [ ]:
nb_epoch = 100
batch_size = 2

history = unet.fit(x_train, y_train, 
                 batch_size=batch_size, 
                 callbacks=my_callbacks,#ayarladığımız callbacklistesini buraya veriyoruz
                 epochs=nb_epoch, verbose=1,
                 validation_data=(x_val, y_val)
                 )



In [ ]:
print("2")

In [ ]:
unet.save_weights("model_weights.hdf5")

## Model testing

results = unet.evaluate(x_test, y_test, batch_size=8)
print("test loss, test acc:", results)

In [ ]:
print(history.history.keys())

## visualization of model accuracy and loss

In [ ]:
plt.plot(history.history['mean_io_u'],color="green")
plt.plot(history.history['val_mean_io_u'],color="blue")
plt.plot(history.history['loss'],color="red")
plt.plot(history.history['val_loss'],color="orange")
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val_acc','loss','val_loss'], loc='upper left')
isim_fig = "unet" + '_fig_acc.png'
plt.savefig(isim_fig)

## Prediction from test images

In [ ]:
sonuc_img = np.zeros((x_test.shape[0],512,512))
deneme = list(range(x_test.shape[0]))
for i in deneme:
    A = np.zeros((1,512,512,1))
    A[0,:,:,:] = x_test[i,:,:,:]
    B = unet.predict(A)
    C = B[:,:,:,0]
    D = C>0.5
    D = D[0,:,:]
    sonuc_img[i,:,:] = D
#     isim = 'sonuc_' + "unet" + '_' +str(i) + '.tif'
#     cv2.imwrite(isim, 255*D)

## Checking whether the predicted images have a zero matrix.

In [ ]:
sayac = 0
for i in range(x_test.shape[0]):
    if np.sum(sonuc_img[i]) == 0:
        sayac += 1
        #print("boş",i)
print(sayac)

##  visualization of predicted images

In [ ]:
plt.imshow(x_test[12,:,:,0],cmap="Greys")
plt.title("x_test")
plt.figure()
plt.imshow(sonuc_img[12,:,:],cmap="Greys")
plt.title("predicted")
plt.figure()
plt.imshow(y_test[12,:,:,0],cmap="Greys")
plt.title("y_test")


In [ ]:
print(sonuc_img.shape)

In [ ]:
label = y_test[:,:,:,0]
index = 72
sonuc = sonuc_img

plt.imshow(label[index],cmap="Greys")
plt.figure()
plt.imshow(sonuc[index],cmap="Greys")
plt.figure()
plt.imshow(x_test[index,:,:,0],cmap="Greys")

## Accuracy Test (IoU and F1 score are most important metrics for segmentation models)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
import warnings


label = label > 0.5


label = label.ravel()


sonuc = sonuc > 0.5

sonuc = sonuc.ravel()

print('Accuracy:')
print(np.mean(label == sonuc))

print('IoU Score (Jaccard Score):')
print(jaccard_score(label, sonuc))

print('F1 Score:')
print(f1_score(label, sonuc))

print('Precision:')
print(precision_score(label, sonuc))

print('Recall:')
print(recall_score(label, sonuc))

In [ ]:
label_list = []
title_list = []
print("predicted\tdata\t\tlabel")
for i in range(12):
    label_list.append(sonuc_img[i+22,:,:])
    title_list.append("predicted")
    label_list.append(x_test[i+22,:,:,0])
    title_list.append("data")
    label_list.append(y_test[i+22,:,:,0])
    title_list.append("label")
  
  

for i in range(6):
    plt.subplot(2,3,i+1)
    #plt.title(title_list[i])
    plt.imshow(label_list[i],cmap="Greys")
    plt.axis('off')
    plt.show()